# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import RidgeCV, LassoCV, LinearRegression
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor
import patsy
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

In [3]:
df = pd.read_csv('previsao_de_renda.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   index                  15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12466 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  mau                    15000 non-null  bool   
 15  re

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [9]:
for col in ['data_ref', 'index']:
    if col in df.columns:
        df.drop(col, axis=1, inplace=True)

df.dropna(inplace=True)
df['log_renda'] = np.log(df['renda'])

formula = 'log_renda ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + tipo_renda + educacao + estado_civil + tipo_residencia + idade + tempo_emprego + qt_pessoas_residencia'
y, X = patsy.dmatrices(formula, df, return_type='dataframe')

X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel(), test_size=0.25, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
ridge = RidgeCV(alphas=alphas, cv=5)
ridge.fit(X_train_scaled, y_train)
ridge_r2 = r2_score(y_test, ridge.predict(X_test_scaled))

print(f'Ridge - Melhor alpha: {ridge.alpha_:.4f} | R² teste: {ridge_r2:.4f}')

lasso = LassoCV(alphas=alphas, cv=5, max_iter=10000)
lasso.fit(X_train_scaled, y_train)
lasso_r2 = r2_score(y_test, lasso.predict(X_test_scaled))

print(f'Lasso - Melhor alpha: {lasso.alpha_:.4f} | R² teste: {lasso_r2:.4f}')

stepwise_X = X.copy()
stepwise_X['Intercept'] = 1
cols = list(stepwise_X.columns)
removed = []
current_score = sm.OLS(y, stepwise_X).fit().rsquared_adj

for col in cols:
    temp_cols = cols.copy()
    temp_cols.remove(col)
    temp_model = sm.OLS(y, stepwise_X[temp_cols]).fit()
    if temp_model.rsquared_adj > current_score:
        cols = temp_cols
        current_score = temp_model.rsquared_adj
        removed.append(col)

final_model = sm.OLS(y, stepwise_X[cols]).fit()
stepwise_r2 = r2_score(y_test, final_model.predict(X_test[cols]))

print(f'Stepwise - R² teste: {stepwise_r2:.4f}')
print("Variáveis removidas:", removed)

df['idade_emprego'] = df['idade'] * df['tempo_emprego']
formula2 = formula + ' + idade_emprego'
y2, X2 = patsy.dmatrices(formula2, df, return_type='dataframe')
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2.values.ravel(), test_size=0.25, random_state=42)
scaler2 = StandardScaler()
X2_train_scaled = scaler2.fit_transform(X2_train)
X2_test_scaled = scaler2.transform(X2_test)
ridge2 = RidgeCV(alphas=alphas, cv=5)
ridge2.fit(X2_train_scaled, y2_train)
ridge2_r2 = r2_score(y2_test, ridge2.predict(X2_test_scaled))

print(f'Ridge com transformação - R² teste: {ridge2_r2:.4f}')

tree = DecisionTreeRegressor(max_depth=5, random_state=42)
tree.fit(X_train, y_train)
tree_r2 = r2_score(y_test, tree.predict(X_test))

print(f'Árvore de Regressão - R² teste: {tree_r2:.4f}')

Ridge - Melhor alpha: 0.1000 | R² teste: 0.2321
Lasso - Melhor alpha: 0.0050 | R² teste: 0.2331


C:\Users\lukas\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:683: UserWarning: Coordinate descent without L1 regularization may lead to unexpected results and is discouraged. Set l1_ratio > 0 to add L1 regularization.
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\lukas\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1793.868349967292, tolerance: 0.4763234866940377
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\lukas\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:683: UserWarning: Coordinate descent without L1 regularization may lead to unexpected results and is discouraged. Set l1_ratio > 0 to add L1 regularization.
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\lukas\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:683: ConvergenceWarning: O

Stepwise - R² teste: 0.1971
Variáveis removidas: ['Intercept', 'posse_de_veiculo[T.True]', 'tipo_renda[T.Bolsista]', 'estado_civil[T.União]']
Ridge com transformação - R² teste: 0.2322
Árvore de Regressão - R² teste: 0.2238
